**Red neuronal feed forward/MLP** - Daniel Conrado Monroy Madrid 16012674

In [1]:
from google.colab import drive
drive.mount('/content/drive/',force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


In [2]:
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score
import pandas_profiling
from sklearn.preprocessing import StandardScaler

Using TensorFlow backend.


In [0]:
def import_data(dataset): #dataset es un string.csv 
  path = "/content/drive/My Drive/datasets/"
  file_name = dataset
  dataset = pd.read_csv(path+file_name)
  return dataset

# Red 1

In [4]:
data1 = import_data('uci_credit_card.csv')
data1.head()

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,BILL_AMT2,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default.payment.next.month
0,1,20000.0,2,2,1,24,2,2,-1,-1,-2,-2,3913.0,3102.0,689.0,0.0,0.0,0.0,0.0,689.0,0.0,0.0,0.0,0.0,1
1,2,120000.0,2,2,2,26,-1,2,0,0,0,2,2682.0,1725.0,2682.0,3272.0,3455.0,3261.0,0.0,1000.0,1000.0,1000.0,0.0,2000.0,1
2,3,90000.0,2,2,2,34,0,0,0,0,0,0,29239.0,14027.0,13559.0,14331.0,14948.0,15549.0,1518.0,1500.0,1000.0,1000.0,1000.0,5000.0,0
3,4,50000.0,2,2,1,37,0,0,0,0,0,0,46990.0,48233.0,49291.0,28314.0,28959.0,29547.0,2000.0,2019.0,1200.0,1100.0,1069.0,1000.0,0
4,5,50000.0,1,2,1,57,-1,0,-1,0,0,0,8617.0,5670.0,35835.0,20940.0,19146.0,19131.0,2000.0,36681.0,10000.0,9000.0,689.0,679.0,0


**Análisis exploratorio, tratamiento y preparación de la data**

In [5]:
pandas_profiling.ProfileReport(data1)

Number of variables,25
Number of observations,30000
Total Missing (%),0.0%
Total size in memory,5.7 MiB
Average record size in memory,200.0 B
Numeric,18
Categorical,0
Boolean,2
Date,0
Text (Unique),0
Rejected,5


In [0]:
features = data1.iloc[:,1:data1.shape[1]-1]
y = data1.iloc[:,-1]   ## target = default.payment.next.month

In [7]:
features.head(3)

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,BILL_AMT2,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6
0,20000.0,2,2,1,24,2,2,-1,-1,-2,-2,3913.0,3102.0,689.0,0.0,0.0,0.0,0.0,689.0,0.0,0.0,0.0,0.0
1,120000.0,2,2,2,26,-1,2,0,0,0,2,2682.0,1725.0,2682.0,3272.0,3455.0,3261.0,0.0,1000.0,1000.0,1000.0,0.0,2000.0
2,90000.0,2,2,2,34,0,0,0,0,0,0,29239.0,14027.0,13559.0,14331.0,14948.0,15549.0,1518.0,1500.0,1000.0,1000.0,1000.0,5000.0


In [8]:
features2 = features[['LIMIT_BAL','AGE','PAY_0','PAY_2','PAY_3','PAY_4','PAY_5','PAY_6','BILL_AMT1','PAY_AMT1','PAY_AMT2','PAY_AMT3','PAY_AMT4','PAY_AMT5','PAY_AMT6']]
features2.head(3) #features que se van a normalizar

,LIMIT_BAL,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6
0,20000.0,24,2,2,-1,-1,-2,-2,3913.0,0.0,689.0,0.0,0.0,0.0,0.0
1,120000.0,26,-1,2,0,0,0,2,2682.0,0.0,1000.0,1000.0,1000.0,0.0,2000.0
2,90000.0,34,0,0,0,0,0,0,29239.0,1518.0,1500.0,1000.0,1000.0,1000.0,5000.0


In [0]:
scaler = StandardScaler()
new_features = pd.DataFrame(scaler.fit_transform(features2),columns=['LIMIT_BAL','AGE','PAY_0','PAY_2','PAY_3','PAY_4','PAY_5','PAY_6','BILL_AMT1','PAY_AMT1','PAY_AMT2','PAY_AMT3','PAY_AMT4','PAY_AMT5','PAY_AMT6'])

In [10]:
other_x = features[['SEX','EDUCATION','MARRIAGE']]
new_data1 = pd.concat([new_features,other_x],axis=1)
new_data1.head(2)

,LIMIT_BAL,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,SEX,EDUCATION,MARRIAGE
0,-1.136720,-1.246020,1.794564,1.782348,-0.696663,-0.666599,-1.530046,-1.486041,-0.642501,-0.341942,-0.227086,-0.296801,-0.308063,-0.314136,-0.293382,2,2,1
1,-0.365981,-1.029047,-0.874991,1.782348,0.138865,0.188746,0.234917,1.992316,-0.659219,-0.341942,-0.213588,-0.240005,-0.244230,-0.314136,-0.180878,2,2,2


In [11]:
#Split data
X_train, X_test, y_train, y_test = train_test_split(new_data1.values, y.values, test_size=0.20, random_state=42)
print('Tamaño X_train: ',X_train.shape)
print('Tamaño y_train: ',y_train.shape)

Tamaño X_train:  (24000, 18)
Tamaño y_train:  (24000,)


In [0]:
def red1(xtrain,ytrain,xdata,ydata,epoch,batch,optimizador):
  model = Sequential()
  model.add(Dense(20, input_dim=18, activation='relu')) #input dim es igual a la cantidad de variables (18)
  model.add(Dense(10, activation='relu'))
  model.add(Dense(1, activation='sigmoid'))
  model.compile(loss='binary_crossentropy', optimizer=optimizador, metrics=['accuracy']) #optimizador es un string
  model.fit(xtrain, ytrain, epochs=epoch, batch_size=batch)
  _, accuracy = model.evaluate(xdata,ydata)
  predicciones = model.predict_classes(xdata)
  precision = precision_score(ydata,predicciones)
  recall = recall_score(ydata,predicciones)
  f1 = f1_score(ydata,predicciones)
  print('F1 score: %.2f' % (f1))
  result = [pd.DataFrame({'F1 score':f1,'Precision':precision,'Recall':recall},index=np.array([1])),model.summary()]
  return result

In [14]:
red1(X_train,y_train,X_train,y_train,150,50,'adam')






Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Epoch 1/150
24000/24000 [==============================] - 7s 286us/step - loss: 0.4756 - acc: 0.8015
Epoch 2/150
24000/24000 [==============================] - 2s 92us/step - loss: 0.4474 - acc: 0.8160
Epoch 3/150
24000/24000 [==============================] - 2s 91us/step - loss: 0.4418 - acc: 0.8175
Epoch 4/150
24000/24000 [==============================] - 2s 95us/step - loss: 0.4380 - acc: 0.8194
Epoch 5/150
24000/24000 [==============================] - 2s 97us/step - loss: 0.4350 - acc: 0.8196
Epoch 6/150
24000/24000 [==============================] - 2s 95us/step - loss: 0.4331 - acc: 0.8201
Epoch 7/150
24000/24000 [==============================] - 2s 91us/step - loss: 0.4311 - acc: 0.8192
Epoch 8/150
24000/24000 [==============================] - 2s 94us/step - loss: 0.4301 - acc: 0.8212
Epoch 9/150
24000/24000 [==============================] - 2s 95us/step - loss: 0.4291 - ac

[   F1 score  Precision    Recall
 1  0.474087   0.713908  0.354875, None]

In [0]:
def red11(xtrain,ytrain,xdata,ydata,epoch,batch,optimizador):
  model = Sequential()
  model.add(Dense(20, input_dim=18, activation='relu')) #input dim es igual a la cantidad de variables (18)
  model.add(Dense(10, activation='relu'))
  model.add(Dense(5, activation='relu'))
  model.add(Dense(1, activation='sigmoid'))
  model.compile(loss='binary_crossentropy', optimizer=optimizador, metrics=['accuracy']) #optimizador es un string
  model.fit(xtrain, ytrain, epochs=epoch, batch_size=batch)
  _, accuracy = model.evaluate(xdata,ydata)
  predicciones = model.predict_classes(xdata)
  precision = precision_score(ydata,predicciones)
  recall = recall_score(ydata,predicciones)
  f1 = f1_score(ydata,predicciones)
  print('F1 score: %.2f' % (f1))
  result = [pd.DataFrame({'F1 score':f1,'Precision':precision,'Recall':recall},index=np.array([1])),model.summary()]
  return result

In [14]:
red11(X_train,y_train,X_train,y_train,200,50,'adam')






Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Epoch 1/200
24000/24000 [==============================] - 7s 284us/step - loss: 0.4812 - acc: 0.7850
Epoch 2/200
24000/24000 [==============================] - 2s 99us/step - loss: 0.4506 - acc: 0.8136
Epoch 3/200
24000/24000 [==============================] - 2s 96us/step - loss: 0.4441 - acc: 0.8168
Epoch 4/200
24000/24000 [==============================] - 2s 97us/step - loss: 0.4413 - acc: 0.8175
Epoch 5/200
24000/24000 [==============================] - 2s 95us/step - loss: 0.4386 - acc: 0.8183
Epoch 6/200
24000/24000 [==============================] - 2s 101us/step - loss: 0.4366 - acc: 0.8200
Epoch 7/200
24000/24000 [==============================] - 2s 99us/step - loss: 0.4345 - acc: 0.8213
Epoch 8/200
24000/24000 [==============================] - 2s 98us/step - loss: 0.4328 - acc: 0.8214
Epoch 9/200
24000/24000 [==============================] - 2s 98us/step - loss: 0.4319 - a

[   F1 score  Precision    Recall
 1  0.498172   0.708984  0.383994, None]

Como se puede observar, la función de costo decrece muy poco en muchas iteraciones. En la siguiente función se experimenta con distintos valores de learning utilizando el algortimo de Nesterov. 

In [0]:
from keras import optimizers

In [0]:
def red12(xtrain,ytrain,xdata,ydata,epoch,batch,learning_rate,momen,nes):
  model = Sequential()
  model.add(Dense(20, input_dim=18, activation='relu')) #input dim es igual a la cantidad de variables (18)
  model.add(Dense(10, activation='relu'))
  model.add(Dense(5, activation='relu'))
  model.add(Dense(1, activation='sigmoid'))
  sgd = optimizers.SGD(lr=learning_rate, momentum=momen, nesterov=nes)
  model.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy']) #optimizador es un string
  model.fit(xtrain, ytrain, epochs=epoch, batch_size=batch)
  _, accuracy = model.evaluate(xdata,ydata)
  predicciones = model.predict_classes(xdata)
  precision = precision_score(ydata,predicciones)
  recall = recall_score(ydata,predicciones)
  f1 = f1_score(ydata,predicciones)
  print('F1 score: %.2f' % (f1))
  result = [pd.DataFrame({'F1 score':f1,'Precision':precision,'Recall':recall},index=np.array([1])),model.summary()]
  return result

In [22]:
red12(X_train,y_train,X_train,y_train,200,50,1,0.3,False) #clasical momentum 

Epoch 1/200
24000/24000 [==============================] - 2s 90us/step - loss: 0.4687 - acc: 0.8009
Epoch 2/200
24000/24000 [==============================] - 2s 79us/step - loss: 0.4549 - acc: 0.8146
Epoch 3/200
24000/24000 [==============================] - 2s 76us/step - loss: 0.4503 - acc: 0.8146
Epoch 4/200
24000/24000 [==============================] - 2s 77us/step - loss: 0.4480 - acc: 0.8156
Epoch 5/200
24000/24000 [==============================] - 2s 78us/step - loss: 0.4450 - acc: 0.8177
Epoch 6/200
24000/24000 [==============================] - 2s 79us/step - loss: 0.4473 - acc: 0.8119
Epoch 7/200
24000/24000 [==============================] - 2s 77us/step - loss: 0.4482 - acc: 0.8146
Epoch 8/200
24000/24000 [==============================] - 2s 78us/step - loss: 0.4464 - acc: 0.8135
Epoch 9/200
24000/24000 [==============================] - 2s 78us/step - loss: 0.4440 - acc: 0.8167
Epoch 10/200
24000/24000 [==============================] - 2s 77us/step - loss: 0.4459 - a

[   F1 score  Precision    Recall
 1  0.504848   0.667387  0.405974, None]

In [24]:
red12(X_train,y_train,X_train,y_train,210,50,0.01,0.9,True) #Nesterov momentum 

Epoch 1/210
24000/24000 [==============================] - 2s 98us/step - loss: 0.5412 - acc: 0.7722
Epoch 2/210
24000/24000 [==============================] - 2s 82us/step - loss: 0.5214 - acc: 0.7782
Epoch 3/210
24000/24000 [==============================] - 2s 84us/step - loss: 0.4708 - acc: 0.7851
Epoch 4/210
24000/24000 [==============================] - 2s 86us/step - loss: 0.4478 - acc: 0.8160
Epoch 5/210
24000/24000 [==============================] - 2s 81us/step - loss: 0.4424 - acc: 0.8176
Epoch 6/210
24000/24000 [==============================] - 2s 86us/step - loss: 0.4386 - acc: 0.8192
Epoch 7/210
24000/24000 [==============================] - 2s 82us/step - loss: 0.4359 - acc: 0.8198
Epoch 8/210
24000/24000 [==============================] - 2s 81us/step - loss: 0.4342 - acc: 0.8212
Epoch 9/210
24000/24000 [==============================] - 2s 82us/step - loss: 0.4333 - acc: 0.8196
Epoch 10/210
24000/24000 [==============================] - 2s 84us/step - loss: 0.4315 - a

[   F1 score  Precision   Recall
 1  0.525527   0.679785  0.42833, None]

In [25]:
red12(X_train,y_train,X_train,y_train,200,50,0.1,0.8,True)

Epoch 1/200
24000/24000 [==============================] - 2s 97us/step - loss: 0.4645 - acc: 0.8087
Epoch 2/200
24000/24000 [==============================] - 2s 81us/step - loss: 0.4510 - acc: 0.8161
Epoch 3/200
24000/24000 [==============================] - 2s 81us/step - loss: 0.4480 - acc: 0.8163
Epoch 4/200
24000/24000 [==============================] - 2s 86us/step - loss: 0.4471 - acc: 0.8162
Epoch 5/200
24000/24000 [==============================] - 2s 82us/step - loss: 0.4430 - acc: 0.8166
Epoch 6/200
24000/24000 [==============================] - 2s 83us/step - loss: 0.4414 - acc: 0.8174
Epoch 7/200
24000/24000 [==============================] - 2s 80us/step - loss: 0.4389 - acc: 0.8178
Epoch 8/200
24000/24000 [==============================] - 2s 79us/step - loss: 0.4375 - acc: 0.8175
Epoch 9/200
24000/24000 [==============================] - 2s 80us/step - loss: 0.4375 - acc: 0.8174
Epoch 10/200
24000/24000 [==============================] - 2s 84us/step - loss: 0.4361 - a

[   F1 score  Precision    Recall
 1  0.513956   0.688763  0.409919, None]

In [26]:
red12(X_train,y_train,X_train,y_train,210,100,0.01,0.9,True)

Epoch 1/210
24000/24000 [==============================] - 1s 59us/step - loss: 0.4739 - acc: 0.7995
Epoch 2/210
24000/24000 [==============================] - 1s 44us/step - loss: 0.4491 - acc: 0.8148
Epoch 3/210
24000/24000 [==============================] - 1s 42us/step - loss: 0.4439 - acc: 0.8165
Epoch 4/210
24000/24000 [==============================] - 1s 41us/step - loss: 0.4412 - acc: 0.8173
Epoch 5/210
24000/24000 [==============================] - 1s 42us/step - loss: 0.4389 - acc: 0.8187
Epoch 6/210
24000/24000 [==============================] - 1s 40us/step - loss: 0.4373 - acc: 0.8171
Epoch 7/210
24000/24000 [==============================] - 1s 41us/step - loss: 0.4362 - acc: 0.8191
Epoch 8/210
24000/24000 [==============================] - 1s 45us/step - loss: 0.4353 - acc: 0.8189
Epoch 9/210
24000/24000 [==============================] - 1s 41us/step - loss: 0.4344 - acc: 0.8194
Epoch 10/210
24000/24000 [==============================] - 1s 40us/step - loss: 0.4330 - a

[   F1 score  Precision    Recall
 1  0.512253   0.698346  0.404471, None]

In [28]:
red12(X_train,y_train,X_train,y_train,300,1000,0.01,0.9,True)

Epoch 1/300
24000/24000 [==============================] - 1s 23us/step - loss: 0.6123 - acc: 0.6864
Epoch 2/300
24000/24000 [==============================] - 0s 6us/step - loss: 0.4954 - acc: 0.7782
Epoch 3/300
24000/24000 [==============================] - 0s 6us/step - loss: 0.4877 - acc: 0.7784
Epoch 4/300
24000/24000 [==============================] - 0s 5us/step - loss: 0.4826 - acc: 0.7787
Epoch 5/300
24000/24000 [==============================] - 0s 5us/step - loss: 0.4775 - acc: 0.7799
Epoch 6/300
24000/24000 [==============================] - 0s 6us/step - loss: 0.4724 - acc: 0.7822
Epoch 7/300
24000/24000 [==============================] - 0s 5us/step - loss: 0.4673 - acc: 0.7901
Epoch 8/300
24000/24000 [==============================] - 0s 5us/step - loss: 0.4626 - acc: 0.8029
Epoch 9/300
24000/24000 [==============================] - 0s 5us/step - loss: 0.4588 - acc: 0.8083
Epoch 10/300
24000/24000 [==============================] - 0s 5us/step - loss: 0.4558 - acc: 0.810

[   F1 score  Precision    Recall
 1  0.492733   0.673396  0.388503, None]

In [29]:
red12(X_train,y_train,X_test,y_test,210,50,0.01,0.9,True) #el mejor 

Epoch 1/210
24000/24000 [==============================] - 2s 104us/step - loss: 0.4651 - acc: 0.8017
Epoch 2/210
24000/24000 [==============================] - 2s 78us/step - loss: 0.4477 - acc: 0.8120
Epoch 3/210
24000/24000 [==============================] - 2s 83us/step - loss: 0.4442 - acc: 0.8140
Epoch 4/210
24000/24000 [==============================] - 2s 82us/step - loss: 0.4413 - acc: 0.8155
Epoch 5/210
24000/24000 [==============================] - 2s 81us/step - loss: 0.4386 - acc: 0.8178
Epoch 6/210
24000/24000 [==============================] - 2s 85us/step - loss: 0.4370 - acc: 0.8181
Epoch 7/210
24000/24000 [==============================] - 2s 79us/step - loss: 0.4355 - acc: 0.8195
Epoch 8/210
24000/24000 [==============================] - 2s 82us/step - loss: 0.4343 - acc: 0.8193
Epoch 9/210
24000/24000 [==============================] - 2s 80us/step - loss: 0.4330 - acc: 0.8195
Epoch 10/210
24000/24000 [==============================] - 2s 79us/step - loss: 0.4322 - 

[   F1 score  Precision    Recall
 1  0.482949   0.611435  0.399086, None]

In [0]:
def red13(xtrain,ytrain,xdata,ydata,epoch,batch,learning_rate,momen,nes):
  model = Sequential()
  model.add(Dense(3, input_dim=18, activation='relu')) #input dim es igual a la cantidad de variables (18)
  model.add(Dense(1, activation='sigmoid'))
  sgd = optimizers.SGD(lr=learning_rate, momentum=momen, nesterov=nes)
  model.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy']) #optimizador es un string
  model.fit(xtrain, ytrain, epochs=epoch, batch_size=batch)
  _, accuracy = model.evaluate(xdata,ydata)
  predicciones = model.predict_classes(xdata)
  precision = precision_score(ydata,predicciones)
  recall = recall_score(ydata,predicciones)
  f1 = f1_score(ydata,predicciones)
  print('F1 score: %.2f' % (f1))
  result = [pd.DataFrame({'F1 score':f1,'Precision':precision,'Recall':recall},index=np.array([1])),model.summary()]
  return result

In [32]:
red13(X_train,y_train,X_test,y_test,300,100,0.001,0.9,True)

Epoch 1/300
24000/24000 [==============================] - 1s 55us/step - loss: 0.5726 - acc: 0.7725
Epoch 2/300
24000/24000 [==============================] - 1s 34us/step - loss: 0.5216 - acc: 0.7782
Epoch 3/300
24000/24000 [==============================] - 1s 35us/step - loss: 0.5052 - acc: 0.7783
Epoch 4/300
24000/24000 [==============================] - 1s 36us/step - loss: 0.4948 - acc: 0.7783
Epoch 5/300
24000/24000 [==============================] - 1s 36us/step - loss: 0.4879 - acc: 0.7783
Epoch 6/300
24000/24000 [==============================] - 1s 33us/step - loss: 0.4831 - acc: 0.7783
Epoch 7/300
24000/24000 [==============================] - 1s 35us/step - loss: 0.4792 - acc: 0.7782
Epoch 8/300
24000/24000 [==============================] - 1s 35us/step - loss: 0.4760 - acc: 0.7783
Epoch 9/300
24000/24000 [==============================] - 1s 34us/step - loss: 0.4733 - acc: 0.7782
Epoch 10/300
24000/24000 [==============================] - 1s 35us/step - loss: 0.4710 - a

[   F1 score  Precision    Recall
 1  0.458706   0.661406  0.351104, None]

Como se puede observar, se utilizaron dos arquitecturas diferentes para probar el desempeño del modelo. En ambos casos el accuracy obtenido fue superior al 80% (en general se mantuvo alrededor del 82%). No obstante, el F1 score no superó el 0.50. Dado el proceso de experimentación, el mejor modelo fue en donde se corrieron 210 iteraciones con batches de 50, learning rate de 0.01, y momentum 0.9 agilizando gradient descent con el algortimo de nesterov. En este caso, el F1 fue cercano al 0.50 (0.48) y el accuracy se mantuvo superior a 0.80. 

Un reto para este problema fue que el error se estancaba en aproximadamente 0.40-0.42. A pesar de experimentar con diferentes arquitecturas y seteo de hiperparámetros, el error nunca redujo su valor. 